**Do datasets work?**
* I think I don't need the `isinstance` dataset with `.copy()` if I can just use the `apply_ufunc`.

**Other stuff**
* Various NaN policies (return NaN, interpolate, skip)

In [1]:
%matplotlib inline
import numpy as np
import xarray as xr
import numpy.polynomial.polynomial as poly
import cftime

In [2]:
from esmtools.utils import convert_time

def _compute_slope(x, y):
    """
    Private function to compute slopes at each grid cell using
    polyfit. 
    """
    return np.polyfit(x, y, 1)[0] # return only the slope

def compute_slope(x, y, dim='time'):
    """
    Computes linear slope (m) at each grid cell.
    
    Args:
      da: xarray DataArray to compute slopes for
      
    Returns:
      xarray DataArray with slopes computed at each grid cell.
    """
    x = convert_time(x, dim)
    
    slopes = xr.apply_ufunc(_compute_slope,
                            x,
                            y,
                            vectorize=True,
                            dask='parallelized', 
                            input_core_dims=[[dim], [dim]],
                            output_dtypes=[float],
                            )
    return slopes

In [15]:
DAYS_PER_MONTH = {
    'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    '365_day': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    'standard': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    'all_leap': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    '366_day': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30],
    'julian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
}

In [21]:
{k: np.sum(v) for (k, v) in DAYS_PER_MONTH.items()}

{'noleap': 365,
 '365_day': 365,
 'standard': 365,
 'gregorian': 365,
 'proleptic_gregorian': 365,
 'all_leap': 366,
 '366_day': 366,
 '360_day': 360,
 'julian': 365}

In [19]:
np.sum(DAYS_PER_MONTH['noleap'])

365

In [11]:
ds = xr.open_dataset('/Users/ribr5703/Desktop/dev/climpred-data/CESM-DP-LE.SST.eastern_pacific.nc')
ds = ds.SST.isel(lead=0).rename({'init': 'time'})

In this case, we give integers for time, so it just computes the delta x equally and returns the slope in units of per year.

In [14]:
ds.time

<xarray.DataArray 'time' (time: 64)>
array([1954., 1955., 1956., 1957., 1958., 1959., 1960., 1961., 1962., 1963.,
       1964., 1965., 1966., 1967., 1968., 1969., 1970., 1971., 1972., 1973.,
       1974., 1975., 1976., 1977., 1978., 1979., 1980., 1981., 1982., 1983.,
       1984., 1985., 1986., 1987., 1988., 1989., 1990., 1991., 1992., 1993.,
       1994., 1995., 1996., 1997., 1998., 1999., 2000., 2001., 2002., 2003.,
       2004., 2005., 2006., 2007., 2008., 2009., 2010., 2011., 2012., 2013.,
       2014., 2015., 2016., 2017.], dtype=float32)
Coordinates:
  * time     (time) float32 1954.0 1955.0 1956.0 1957.0 ... 2015.0 2016.0 2017.0
    lead     int32 1
Attributes:
    long_name:    ensemble
    description:  historical year corresponding to forecast year 1
    example:      S=1955 for forecasts initialized on November 1 1954

In [4]:
A = compute_slope(ds.time, ds)

`numpy` datetime

In [5]:
ds = xr.open_dataset('/Users/ribr5703/Desktop/dev/climpred-data/CESM-DP-LE.SST.eastern_pacific.nc')
ds = ds.SST.isel(lead=0).rename({'init': 'time'})
ds['time'] = np.arange('1900-12-31', '1964-12-31', dtype='datetime64[Y]')

In [6]:
B = compute_slope(ds.time, ds)

`cftime` datetime

In [ ]:
ds = xr.open_dataset('/Users/ribr5703/Desktop/dev/climpred-data/CESM-DP-LE.SST.eastern_pacific.nc')
ds = ds.SST.isel(lead=0).rename({'init': 'time'})
ds['time'] = xr.cftime_range('1900', '1964', freq='Y')

In [ ]:
C = compute_slope(ds.time, ds)

In [ ]:
assert (isinstance(ds.time, xr.CFTimeIndex) or np.issubdtype(ds.time, np.datetime64)), f'{type(test)} is not a datetime index'